In [4]:
using CmdStan, Distributions, Dates, Plots, Plots.PlotMeasures
using DataFrames, CSV, Query

In [5]:
ENV["JULIA_CMDSTAN_HOME"]="/Users/marlinfiggins/cmdstan/"
CmdStan.set_cmdstan_home!(ENV["JULIA_CMDSTAN_HOME"])

"/Users/marlinfiggins/cmdstan/"

In [6]:
run_date = "today"

"09_23_21"

In [7]:
# What is proper way to load module?
include("../src/BayesTS/BayesTS.jl")
using .BayesTS

In [8]:
# Load helper functions 
include("../src/BayesTShelpers.jl")
include("../src/DataHelpers.jl")
include("../src/InferenceHelpers.jl")
include("../src/lifetimes.jl")

add_other_parms (generic function with 2 methods)

# Load Data

In [16]:
df = DataFrame(CSV.File("../data/raw/cases-lineage-sequence-counts-2021-09-23.tsv"))

,state,date,cases,sequences_total,sequences_other,sequences_B.1.1.7
,String,Date,Int64,Int64,Int64,Int64
1,Alabama,2021-01-28,2340,0,0,0
2,Alabama,2021-01-29,1967,5,5,0
3,Alabama,2021-01-30,1253,0,0,0
4,Alabama,2021-01-31,1228,1,1,0
5,Alabama,2021-02-01,2476,15,13,1
6,Alabama,2021-02-02,1997,17,17,0
7,Alabama,2021-02-03,1742,1,0,1
8,Alabama,2021-02-04,1649,2,2,0
9,Alabama,2021-02-05,1582,39,37,1


# Model set up

In [17]:
g = generation_time(20, 5.0, 0.3)
onset = onset_time(20, 4.5, 0.6, 0.8)

20-element Vector{Float64}:
 0.15422107788295136
 0.5979776824957601
 0.8
 0.7462640695453
 0.6021072988929602
 0.45527929483634866
 0.33419359901772533
 0.24227494594996882
 0.175034123856333
 0.12664182163034848
 0.09201514722705773
 0.06723952335640869
 0.049455997273909
 0.0366271997771212
 0.02731671924777422
 0.020515169042784314
 0.015512579078432101
 0.011807873345391351
 0.009045677113215203
 0.006972500059309538

In [18]:
num_SoI = size(filter(row -> row.state == "Washington", df), 1)
seed_L = 14
forecast_L = 0

0

In [19]:
# Choose Basis Model
model = SplineTrend(20,  num_SoI, 4)
prior_model = [["LAS"]]

1-element Vector{Vector{String}}:
 ["LAS"]

In [20]:
states_dict = process_all_states("../src/stan_models/Rt_Structured_lineage.stan", df, 
    g, onset, 
    seed_L, forecast_L, 
    model; priors = prior_model, 
    model_name = "test")

Dict{Any, Any} with 46 entries:
  "Colorado"       => Dict{String, Any}("seq_labels"=>SubString{String}["B.1.1.…
  "Texas"          => Dict{String, Any}("seq_labels"=>SubString{String}["B.1.1.…
  "Alabama"        => Dict{String, Any}("seq_labels"=>SubString{String}["B.1.1.…
  "Vermont"        => Dict{String, Any}("seq_labels"=>SubString{String}["B.1.1.…
  "Missouri"       => Dict{String, Any}("seq_labels"=>SubString{String}["B.1.1.…
  "Wyoming"        => Dict{String, Any}("seq_labels"=>SubString{String}["B.1.1.…
  "California"     => Dict{String, Any}("seq_labels"=>SubString{String}["B.1.1.…
  "Maine"          => Dict{String, Any}("seq_labels"=>SubString{String}["B.1.1.…
  "New_Mexico"     => Dict{String, Any}("seq_labels"=>SubString{String}["B.1.1.…
  "Tennessee"      => Dict{String, Any}("seq_labels"=>SubString{String}["B.1.1.…
  "Nevada"         => Dict{String, Any}("seq_labels"=>SubString{String}["B.1.1.…
  "Kentucky"       => Dict{String, Any}("seq_labels"=>SubString{String}["B.1.

# Run stan models and export parameters

In [21]:
# Get Rt by state
lQuants = [0.25 0.1 0.025]
uQuants = [0.75 0.9 0.975]

df_list = []

Any[]

In [ ]:
states_to_run = sort(collect(keys(states_dict)))
#states_to_run = ["California", "Texas"]
# Run states and export Rt
for state in states_to_run
    run_stan_model!(state, states_dict)
    state_rt = get_Rt_by_state(state, states_dict)
    CSV.write("../data/sims/results/$(run_date)/inferred_lineage_rts_$(state)_$(run_date).csv", state_rt)
    print("Saved state $(state) dataframe")
    push!(df_list, state_rt)
end

In [ ]:
# Export all states Rt as dataframe
final_df = vcat(df_list...)

In [ ]:
# Uncomment to re-run broken states
#final_df = DataFrame(CSV.File("../data/sims/results/09_23_21/inferred_lineage_rts_09_23_2021.csv"))
#final_df = filter(row -> !(row.state in states_to_run), final_df)
#final_df = vcat(df_list..., final_df)

In [ ]:
CSV.write("../data/sims/results/$(run_date)/inferred_lineage_rts_$(run_date).csv", final_df)
CSV.write("../data/sims/results/$(run_date)/inferred_lineage_rts_$(run_date).tsv", delim ="\t", final_df)

In [ ]:
growth_dfs = [get_growth_advantages(SoI, states_dict) for SoI in states_to_run]
final_growth_df = vcat(growth_dfs...)

In [ ]:
CSV.write("../data/sims/results/$(run_date)/inferred_lineage_growth_advantage_$(run_date).csv", final_growth_df)
CSV.write("../data/sims/results/$(run_date)/inferred_lineage_growth_advantage_$(run_date).tsv", delim ="\t", final_growth_df)